# Exercise

My step by step solutions enriched by [Saul's notes](https://drive.google.com/drive/folders/1EF6HW5NL9cM1hWLYa7bVL0bgMzEoqSBC)
Build a *blast-based* method to predict the presence of BPTI/Kunitz domain
in proteins available in SwissProt using the human proteins as a reference.
* Select all Proteins in SwissProt with BPTI/Kunitz domain.
* Separate human from non human proteins. Use the non human proteins
as a positive in the testing set.
* Generate a random set of negative of the same size of the positive set.
* Remove both positives and negatives from SwissProt and perform the
prediction based on the results of the blast search. 

checking the files we downloaded:

## The idea:
1. In swissprot: find all (human + non-human) Kunitz-containing proteins.
2. use the **non-human** fraction as a **positive training set**
3. Generate a **random** set of proteins as a negative set that has the same size as the positive set.
4. Then remove the training set and do a prediction based on a Blast search.

1. **Find human and non-human Kunitz proteins in SwissProt**
Searching all the human proteins in SwissProt containing the Kunitz domain.

`reviewed:yes organism:"Homo sapiens (Human) [9606]" database:(type:pfam PF00014)`

Retrieved 18 proteins. Download them in Fasta format.

In [1]:
grep '>'  Human_PF00014.fasta |wc

      18     200    1778


In [2]:
head Human_PF00014.fasta

>sp|Q06481|APLP2_HUMAN Amyloid-like protein 2 OS=Homo sapiens OX=9606 GN=APLP2 PE=1 SV=2
MAATGTAAAAATGRLLLLLLVGLTAPALALAGYIEALAANAGTGFAVAEPQIAMFCGKLN
MHVNIQTGKWEPDPTGTKSCFETKEEVLQYCQEMYPELQITNVMEANQRVSIDNWCRRDK
KQCKSRFVTPFKCLVGEFVSDVLLVPEKCQFFHKERMEVCENHQHWHTVVKEACLTQGMT
LYSYGMLLPCGVDQFHGTEYVCCPQTKIIGSVSKEEEEEDEEEEEEEDEEEDYDVYKSEF
PTEADLEDFTEAAVDEDDEDEEEGEEVVEDRDYYYDTFKGDDYNEENPTEPGSDGTMSDK
EITHDVKAVCSQEAMTGPCRAVMPRWYFDLSKGKCVRFIYGGCGGNRNNFESEDYCMAVC
KAMIPPTPLPTNDVDVYFETSADDNEHARFQKAKEQLEIRHRNRMDRVKKEWEEAELQAK
NLPKAERQTLIQHFQAMVKALEKEAASEKQQLVETHLARVEAMLNDRRRMALENYLAALQ
SDPPRPHRILQALRRYVRAENKDRLHTIRHYQHVLAVDPEKAAQMKSQVMTHLHVIEERR


Now I download all **non-Human** proteins that contain the **Kunitz Domain**: non human pf00014 file.

`database:(type:pfam pf00014) NOT organism:"Homo sapiens (Human) [9606]" AND reviewed:yes`

lets check how many sequences the fasta file contains.

In [4]:
grep '>' NotHuman_PF00014.fasta |wc

     341    3479   35934


In [5]:
head NotHuman_PF00014.fasta

>sp|P08592|A4_RAT Amyloid-beta A4 protein OS=Rattus norvegicus OX=10116 GN=App PE=1 SV=2
MLPSLALLLLAAWTVRALEVPTDGNAGLLAEPQIAMFCGKLNMHMNVQNGKWESDPSGTK
TCIGTKEGILQYCQEVYPELQITNVVEANQPVTIQNWCKRGRKQCKTHTHIVIPYRCLVG
EFVSDALLVPDKCKFLHQERMDVCETHLHWHTVAKETCSEKSTNLHDYGMLLPCGIDKFR
GVEFVCCPLAEESDSIDSADAEEDDSDVWWGGADTDYADGGEDKVVEVAEEEEVADVEEE
EAEDDEDVEDGDEVEEEAEEPYEEATERTTSIATTTTTTTESVEEVVREVCSEQAETGPC
RAMISRWYFDVTEGKCAPFFYGGCGGNRNNFDTEEYCMAVCGSVSSQSLLKTTSEPLPQD
PVKLPTTAASTPDAVDKYLETPGDENEHAHFQKAKERLEAKHRERMSQVMREWEEAERQA
KNLPKADKKAVIQHFQEKVESLEQEAANERQQLVETHMARVEAMLNDRRRLALENYITAL
QAVPPRPHHVFNMLKKYVRAEQKDRQHTLKHFEHVRMVDPKKAAQIRSQVMTHLRVIYER


These two sets are used as a positives set. The `Human_PF00014.fasta` as training positive and the the `NotHuman_PF00014.fasta` as testing positive set. 


And then we need a set that are for the **negatives**. So a set that are only Human but NOT PF00014 kunitz this dataset contains more than 20 000.

`NOT database:(type:pfam pf00014) AND organism:"Homo sapiens (Human) [9606]" AND reviewed:yes`

In [6]:
 grep '>' Human_NotPF00014.fasta |wc

   20347  222853 2042133


In [7]:
head Human_NotPF00014.fasta

>sp|Q8N8B7|TEANC_HUMAN Transcription elongation factor A N-terminal and central domain-containing protein OS=Homo sapiens OX=9606 GN=TCEANC PE=1 SV=2
MSDKNQIAARASLIEQLMSKRNFEDLGNHLTELETIYVTKEHLQETDVVRAVYRVLKNCP
SVALKKKAKCLLSKWKAVYKQTHSKARNSPKLFPVRGNKEENSGPSHDPSQNETLGICSS
NSLSSQDVAKLSEMIVPENRAIQLKPKEEHFGDGDPESTGKRSSELLDPTTPMRTKCIEL
LYAALTSSSTDQPKADLWQNFAREIEEHVFTLYSKNIKKYKTCIRSKVANLKNPRNSHLQ
QNLLSGTTSPREFAEMTVMEMANKELKQLRASYTESCIQEHYLPQVIDGTQTNKIKCRRC
EKYNCKVTVIDRGTLFLPSWVRNSNPDEQMMTYVICNECGEQWYHSKWVCW
>sp|O75494|SRS10_HUMAN Serine/arginine-rich splicing factor 10 OS=Homo sapiens OX=9606 GN=SRSF10 PE=1 SV=1
MSRYLRPPNTSLFVRNVADDTRSEDLRREFGRYGPIVDVYVPLDFYTRRPRGFAYVQFED
VRDAEDALHNLDRKWICGRQIEIQFAQGDRKTPNQMKAKEGRNVYSSSRYDDYDRYRRSR


## Creating Training and Testing Sets

1. Splitting the Human_NotPF00014.fasta File
Now if you want to split the dataset into 2 parts. Lets take the first 10000:

In [8]:
grep '>' Human_NotPF00014.fasta |wc -l

   20347


In [7]:
 grep '>' Human_NotPF00014.fasta |head -n 10000 |wc

   10000  109297 1005583


Then we can find the 'name' or ID of the 10 000st sequence:
By piping;

`head -- display first lines of a file`
`SYNOPSIS
     head [-n count | -c bytes] [file ...]`

`DESCRIPTION
     This filter displays the first count lines or bytes of each of the specified files, or
     of the standard input if no files are specified.  If count is omitted it defaults to
     10.` 
`tail -- display the last part of a file`
` -n number`

So we pipe the head of - n 10000 into | tail -n 1. That way we get the last of the list.

In [8]:
grep '>' Human_NotPF00014.fasta |head -n 10000 |tail -n 1

>sp|O95081|AGFG2_HUMAN Arf-GAP domain and FG repeat-containing protein 2 OS=Homo sapiens OX=9606 GN=AGFG2 PE=1 SV=2


Now that we have the name of the 10000st seq - we want to find the in which line number it is located.

In [9]:
grep -n O95081 Human_NotPF00014.fasta 

109065:>sp|O95081|AGFG2_HUMAN Arf-GAP domain and FG repeat-containing protein 2 OS=Homo sapiens OX=9606 GN=AGFG2 PE=1 SV=2


The sequence is found in the line `109065`. We can also retrieve the 10th line after the match:

In [11]:
grep -n -A 10 O95081 Human_NotPF00014.fasta #|less

109065:>sp|O95081|AGFG2_HUMAN Arf-GAP domain and FG repeat-containing protein 2 OS=Homo sapiens OX=9606 GN=AGFG2 PE=1 SV=2
109066-MVMAAKKGPGPGGGVSGGKAEAEAASEVWCRRVRELGGCSQAGNRHCFECAQRGVTYVDI
109067-TVGSFVCTTCSGLLRGLNPPHRVKSISMTTFTEPEVVFLQSRGNEVCRKIWLGLFDARTS
109068-LVPDSRDPQKVKEFLQEKYEKKRWYVPPDQVKGPTYTKGSASTPVQGSIPEGKPLRTLLG
109069-DPAPSLSVAASTSSQPVSQSHARTSQARSTQPPPHSSVKKASTDLLADIGGDPFAAPQMA
109070-PAFAAFPAFGGQTPSQGGFANFDAFSSGPSSSVFGSLPPAGQASFQAQPTPAGSSQGTPF
109071-GATPLAPASQPNSLADVGSFLGPGVPAAGVPSSLFGMAGQVPPLQSVTMGGGGGSSTGLA
109072-FGAFTNPFTAPAAQSPLPSTNPFQPNGLAPGPGFGMSSAGPGFPQAVPPTGAFASSFPAP
109073-LFPPQTPLVQQQNGSSFGDLGSAKLGQRPLSQPAGISTNPFMTGPSSSPFASKPPTTNPF
109074-L
109075->sp|P32970|CD70_HUMAN CD70 antigen OS=Homo sapiens OX=9606 GN=CD70 PE=1 SV=2


Knowing that one entry 'spans over' 10 lines - line one being the identifier and the remaining ones the sequence, we also know that we have to add 9 lines in order to 'keep' the whole sequence. As not to trucate it keeping only line one of it. The last line shold therefore be 109074.

(Or an easier way to approach it would have been to access sequence 100001 and find the number of the previous line...)

To check again that we calculated it right and we will obtain the entire seq of the sp|095081 we use head again: 

In [12]:
head -n 109074 Human_NotPF00014.fasta |tail -n 10

>sp|O95081|AGFG2_HUMAN Arf-GAP domain and FG repeat-containing protein 2 OS=Homo sapiens OX=9606 GN=AGFG2 PE=1 SV=2
MVMAAKKGPGPGGGVSGGKAEAEAASEVWCRRVRELGGCSQAGNRHCFECAQRGVTYVDI
TVGSFVCTTCSGLLRGLNPPHRVKSISMTTFTEPEVVFLQSRGNEVCRKIWLGLFDARTS
LVPDSRDPQKVKEFLQEKYEKKRWYVPPDQVKGPTYTKGSASTPVQGSIPEGKPLRTLLG
DPAPSLSVAASTSSQPVSQSHARTSQARSTQPPPHSSVKKASTDLLADIGGDPFAAPQMA
PAFAAFPAFGGQTPSQGGFANFDAFSSGPSSSVFGSLPPAGQASFQAQPTPAGSSQGTPF
GATPLAPASQPNSLADVGSFLGPGVPAAGVPSSLFGMAGQVPPLQSVTMGGGGGSSTGLA
FGAFTNPFTAPAAQSPLPSTNPFQPNGLAPGPGFGMSSAGPGFPQAVPPTGAFASSFPAP
LFPPQTPLVQQQNGSSFGDLGSAKLGQRPLSQPAGISTNPFMTGPSSSPFASKPPTTNPF
L


This way we have reconfirmed that the index of the last line we plan to grab is correct. Now we will save the sto of the following cmd to Humand_NotPF00014_Training.

### Now we actually split the set into 2:
in the outcome we want to keep

* 10000 proteins in the Training set - goes until line 109074
* 10347 in the Testing set - starts at line 109075

Generating the **Training set** :

In [13]:
head -n 109074 Human_NotPF00014.fasta > Human_NotPF00014_Training.fasta

Then we select the remaining ones as the **Testing set** by just modifying the cmd:

In [17]:
tail -n +109075 Human_NotPF00014.fasta > Human_NotPF00014_Testing.fasta

As always now we check weather the the files 'look' as we expect them to look:

In [18]:
grep '^>' Human_NotPF00014_Training.fasta |wc -l

   10000


In [19]:
grep '^>' Human_NotPF00014_Testing.fasta |wc -l

   10347


Prof Capriotti actually used:

`grep '>' Human_NotPF00014_Testing.fasta |wc
   10347  113556 1036550`
   


### Training and testing for the positives

In [18]:
# cat NotHuman_PF00014.fasta 

In [8]:
grep '>' NotHuman_PF00014.fasta |wc

     341    3479   35934


Find Uniport id of the 171th seq:

In [10]:
grep '>' NotHuman_PF00014.fasta |head -n 171 |tail -n 1

>sp|Q29428|TKDP1_SHEEP Trophoblast Kunitz domain protein 1 OS=Ovis aries OX=9940 GN=TKDP1 PE=2 SV=1


Find the last line of the seq by looking at that part in the file:
Last line of seq Q29428 is 928. So I will split the file here.

In [7]:
grep -n -A 11 Q29428 NotHuman_PF00014.fasta |cat

923:>sp|Q29428|TKDP1_SHEEP Trophoblast Kunitz domain protein 1 OS=Ovis aries OX=9940 GN=TKDP1 PE=2 SV=1
924-MRQLCLSTALLFLLVILVDSTPLNIYHIQDEGLETSHRRGPEKRSVIDVVSGIINGVATG
925-TKIIEKGAGILTGLAEIITKAIKGQVMISRIQFDNHTLEELPTLQLEYSTLSEENNGLKT
926-SHRRGLEKRSVTDVVTSIINGVATGTKIIEKGAGILTGLAEIITKAIKGQVMISGIQFDN
927-HTLEEYQTLKIEYSTLSEENKASKPALCLEPKVTGDCNATMTRYFYNTQTGLCEQFVYTG
928-CEGNGNNFENLEDCMKTCSQEAGSL
929->sp|Q9D263|SPIT4_MOUSE Kunitz-type protease inhibitor 4 OS=Mus musculus OX=10090 GN=Spint4 PE=2 SV=1
930-MKPTKLGFLLGLSLLCSLSPPVLSGVERLANYLCKDYNDPCLLDVEPGSCYEVHFRFFYN
931-QTAKQCQIFLFTGCNGNLNNFKLKIDCDVTCHETYKNPPLPSGDKRKRSLRALATKNLAT
932-DWRLTTLQPGRLRQTERKETLLQRARRESRRQPLARHHT
933->sp|B5L5R4|IVBI4_NOTSC Tigerin-4 OS=Notechis scutatus scutatus OX=70142 PE=3 SV=1
934-MSSGGLLLLLGLLTLWAELTPVSSKDRPKFCELPADSGPCRGILRAFYYHPVHRTCQMFI


Another way to find the right line to split on.

In [15]:
grep '>' NotHuman_PF00014.fasta -m 171 -n|tail -1
grep '>' NotHuman_PF00014.fasta -m 172 -n| tail -1

923:>sp|Q29428|TKDP1_SHEEP Trophoblast Kunitz domain protein 1 OS=Ovis aries OX=9940 GN=TKDP1 PE=2 SV=1
929:>sp|Q9D263|SPIT4_MOUSE Kunitz-type protease inhibitor 4 OS=Mus musculus OX=10090 GN=Spint4 PE=2 SV=1


### Splitting the positive not humansset into 2:
in the outcome we want to keep

* 171 proteins in the Training set - goes until line 928
* 170 in the Testing set - starts at line 929

Generating the **Training set** :

In [17]:
head -n 928 NotHuman_PF00014.fasta  > NotHuman_PF00014_Training.fasta
tail  -n +929 NotHuman_PF00014.fasta  > NotHuman_PF00014_Testing.fasta
grep ">" NotHuman_PF00014_Training.fasta|tail -1
cat NotHuman_PF00014_Testing.fasta|head -1

>sp|Q29428|TKDP1_SHEEP Trophoblast Kunitz domain protein 1 OS=Ovis aries OX=9940 GN=TKDP1 PE=2 SV=1
>sp|Q9D263|SPIT4_MOUSE Kunitz-type protease inhibitor 4 OS=Mus musculus OX=10090 GN=Spint4 PE=2 SV=1


## Creating the BLAST database

For the human proteins set with the new version of BLAST
Now we have to format the Database:
If you want to format a database
The DB now uses as a reference the human BPTI domain. `formatdb -i <file name> -p T`

Problem on my machine the formatdb is deprecated... instead I had to use:

See: https://www.ncbi.nlm.nih.gov/books/NBK279690/ 

In [10]:
#makeblastdb -in Human_PF00014.fasta -taxid 9606 -dbtype prot

In [11]:
#makeblastdb -in Human_PF00014.fasta -dbtype prot



Building a new DB, current time: 04/21/2020 15:47:52
New DB name:   /Users/ila/01-Unibo/2_LAB1_2nd/capriotti/repkunitz/Human_PF00014.fasta
New DB title:  Human_PF00014.fasta
Sequence type: Protein
Deleted existing Protein BLAST database named /Users/ila/01-Unibo/2_LAB1_2nd/capriotti/repkunitz/Human_PF00014.fasta
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 18 sequences in 0.0606122 seconds.


`Building a new DB, current time: 04/16/2020 18:24:24
New DB name:   /Users/ila/01-Unibo/2_LAB1_2nd/capriotti/repkunitz/Human_PF00014.fasta
New DB title:  Human_PF00014.fasta
Sequence type: Protein
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 18 sequences in 0.0149059 seconds.`

In [12]:
ls

Exercise_Report.ipynb           Human_PF00014.fasta.phr
Human_NotPF00014.fasta          Human_PF00014.fasta.pin
Human_NotPF00014_Testing.fasta  Human_PF00014.fasta.psq
Human_NotPF00014_Training.bl6   NotHuman_PF00014.bl6
Human_NotPF00014_Training.fasta NotHuman_PF00014.bl6.best
Human_PF00014.bl6               NotHuman_PF00014.fasta
Human_PF00014.bl8               bl8
Human_PF00014.fasta             kunitz_meeting.rtf


`ila@Immanuelas-Air:~/01-Unibo/2_LAB1_2nd/capriotti/repkunitz$ ls
13L_lab1-15-04_all.mp4          Human_PF00014.fasta             NotHuman_PF00014.fasta
Human_NotPF00014.fasta          Human_PF00014.fasta.phr         ncbi-blast-2.2.26+.dmg
Human_NotPF00014_Testing.fasta  Human_PF00014.fasta.pin         not_not.fasta
Human_NotPF00014_Training.fasta Human_PF00014.fasta.psq`

Now just to check, we will do the blast of the 
human proteins against the Human proteins just as a consistency test. The evalue was increased just to see what happenes.
`#replacing blastpgp from version 2.2.26 (legacy) -i = -query, -d = -db, -o = -out, -m = -outfmt. `


In [33]:
# outfmt 6 is the Tabular form
#psiblast -query Human_PF00014.fasta -db Human_PF00014.fasta -out Human_PF00014.bl6 -outfmt 6
blastp -query Human_PF00014.fasta -db Human_PF00014.fasta -out autotest.bltab -outfmt 6 -evalue 1000 # -m 8 of the old version7
blastp -query Human_PF00014.fasta -db Human_PF00014.fasta -out Human_PF00014.bl6 -outfmt 6 -evalue 1000 #

The output was redirected into the `Human_PF00014.bl6` file
looking like that:

In [32]:
head Human_PF00014.bl6

sp|Q06481|APLP2_HUMAN	sp|Q06481|APLP2_HUMAN	100.000	763	0	0	1	763	1	763	0.0	1590
sp|Q06481|APLP2_HUMAN	sp|P05067|A4_HUMAN	51.111	495	178	11	313	762	294	769	1.73e-155	463
sp|Q06481|APLP2_HUMAN	sp|O43291|SPIT2_HUMAN	41.667	84	47	1	288	369	111	194	8.02e-21	84.0
sp|Q06481|APLP2_HUMAN	sp|O43291|SPIT2_HUMAN	40.625	64	34	2	298	360	28	88	2.75e-14	64.7
sp|Q06481|APLP2_HUMAN	sp|O43291|SPIT2_HUMAN	25.806	31	19	1	108	134	154	184	169	15.8
sp|Q06481|APLP2_HUMAN	sp|O43291|SPIT2_HUMAN	62.500	8	3	0	539	546	225	232	307	15.0
sp|Q06481|APLP2_HUMAN	sp|O43291|SPIT2_HUMAN	33.333	18	10	1	193	210	125	140	932	13.5
sp|Q06481|APLP2_HUMAN	sp|P49223|SPIT3_HUMAN	47.170	53	28	0	309	361	35	87	5.02e-16	65.1
sp|Q06481|APLP2_HUMAN	sp|P49223|SPIT3_HUMAN	29.412	17	12	0	76	92	71	87	171	15.4
sp|Q06481|APLP2_HUMAN	sp|P49223|SPIT3_HUMAN	80.000	5	1	0	621	625	39	43	362	14.2


Both top and bottom show lines for which the first 2 cols are equal which means that they are matching against them selves.

In [31]:
awk '{if ($1 == $2) print $0}' autotest.bltab 

sp|Q06481|APLP2_HUMAN	sp|Q06481|APLP2_HUMAN	100.000	763	0	0	1	763	1	763	0.0	1590
sp|P05067|A4_HUMAN	sp|P05067|A4_HUMAN	100.000	770	0	0	1	770	1	770	0.0	1610
sp|Q02388|CO7A1_HUMAN	sp|Q02388|CO7A1_HUMAN	100.000	2944	0	0	1	2944	1	2944	0.0	5494
sp|P12111|CO6A3_HUMAN	sp|P12111|CO6A3_HUMAN	100.000	3177	0	0	1	3177	1	3177	0.0	6455
sp|P02760|AMBP_HUMAN	sp|P02760|AMBP_HUMAN	100.000	352	0	0	1	352	1	352	0.0	734
sp|O95925|EPPI_HUMAN	sp|O95925|EPPI_HUMAN	100.000	133	0	0	1	133	1	133	8.42e-100	274
sp|P10646|TFPI1_HUMAN	sp|P10646|TFPI1_HUMAN	100.000	304	0	0	1	304	1	304	0.0	630
sp|O43278|SPIT1_HUMAN	sp|O43278|SPIT1_HUMAN	100.000	529	0	0	1	529	1	529	0.0	1099
sp|O43291|SPIT2_HUMAN	sp|O43291|SPIT2_HUMAN	100.000	252	0	0	1	252	1	252	0.0	521
sp|P48307|TFPI2_HUMAN	sp|P48307|TFPI2_HUMAN	100.000	235	0	0	1	235	1	235	2.53e-180	486
sp|Q8TEU8|WFKN2_HUMAN	sp|Q8TEU8|WFKN2_HUMAN	100.000	576	0	0	1	576	1	576	0.0	1199
sp|Q2UY09|COSA1_HUMAN	sp|Q2UY09|COSA1_HUMAN	100.000	1125	0	0	1	1125	1	1125	0.0	2187
sp|Q96NZ8|WFKN1_HUMAN	

In [ ]:
# sp|Q06481|APLP2_HUMAN   sp|Q06481|APLP2_HUMAN   100.000 763     0       0       1       763     1       763     0.0     1590
# sp|Q06481|APLP2_HUMAN   sp|P05067|A4_HUMAN      51.111  495     178     11      313     762     294     769     1.73e-155       463
# sp|Q06481|APLP2_HUMAN   sp|O43291|SPIT2_HUMAN   41.667  84      47      1       288     369     111     194     8.02e-21        84.0
# sp|Q06481|APLP2_HUMAN   sp|O43291|SPIT2_HUMAN   40.625  64      34      2       298     360     28      88      2.75e-14        64.7
# sp|Q06481|APLP2_HUMAN   sp|P49223|SPIT3_HUMAN   47.170  53      28      0       309     361     35      87      5.02e-16        65.1

The 11th column shows the e-value. To find the highest (=worst) e-value we use reverse sort by 11th column. `-nrk n- number, r- reverse, k- column`

In [19]:
sort -nrk 11 Human_PF00014.bl8 |head #|less

sp|Q9BQY6|WFDC6_HUMAN	sp|Q02388|CO7A1_HUMAN	48.000	25	11	2	12	35	2315	2338	10.0	16.9
sp|P10646|TFPI1_HUMAN	sp|P12111|CO6A3_HUMAN	45.098	51	28	0	217	267	3112	3162	9.99e-12	56.6
sp|P02760|AMBP_HUMAN	sp|Q9BQY6|WFDC6_HUMAN	59.259	27	11	0	256	282	102	128	9.98e-06	35.8
sp|O95428|PPN_HUMAN	sp|P02760|AMBP_HUMAN	38.462	52	32	0	754	805	231	282	9.96e-10	52.4
sp|Q8TEU8|WFKN2_HUMAN	sp|O43278|SPIT1_HUMAN	54.545	44	20	0	393	436	398	441	9.95e-11	54.7
sp|Q02388|CO7A1_HUMAN	sp|P12111|CO6A3_HUMAN	23.721	215	149	8	1025	1233	12	217	9.92e-06	41.6
sp|Q96NZ8|WFKN1_HUMAN	sp|P10646|TFPI1_HUMAN	29.545	132	59	1	313	410	137	268	9.91e-16	68.9
sp|P12111|CO6A3_HUMAN	sp|Q2UY09|COSA1_HUMAN	24.828	145	96	3	1639	1778	798	934	9.90e-06	41.6
sp|Q8IUA0|WFDC8_HUMAN	sp|P12111|CO6A3_HUMAN	66.667	6	2	0	153	158	2395	2400	9.8	18.1
sp|P12111|CO6A3_HUMAN	sp|Q8TEU8|WFKN2_HUMAN	28.788	66	47	0	2897	2962	266	331	9.8	21.9


We see the worst value belongs to Q9BQY6

In [ ]:
#sp|Q9BQY6|WFDC6_HUMAN   sp|Q02388|CO7A1_HUMAN   48.000  25      11      2       12      35      2315    2338    10.0    16.9

We can also change the evalue to 1000 to play:

In [20]:
blastp -query Human_PF00014.fasta -db Human_PF00014.fasta -out Human_PF00014.bl6 -outfmt 6 -evalue 1000

In [21]:
#sort -nrk 11 Human_PF00014.bl8 |less
sort -nrk 11 Human_PF00014.bl6 |head

sp|P48307|TFPI2_HUMAN	sp|P05067|A4_HUMAN	27.273	11	8	0	154	164	551	561	999	11.5
sp|P05067|A4_HUMAN	sp|P12111|CO6A3_HUMAN	22.222	45	33	1	251	295	1752	1794	999	13.5
sp|O95925|EPPI_HUMAN	sp|O43278|SPIT1_HUMAN	30.000	10	7	0	56	65	432	441	999	10.8
sp|Q8TEU8|WFKN2_HUMAN	sp|Q02388|CO7A1_HUMAN	37.500	16	10	0	97	112	2341	2356	998	13.1
sp|Q2UY09|COSA1_HUMAN	sp|Q02388|CO7A1_HUMAN	54.545	11	5	0	495	505	932	942	998	13.9
sp|Q2UY09|COSA1_HUMAN	sp|P12111|CO6A3_HUMAN	50.000	10	5	0	279	288	134	143	998	13.9
sp|Q02388|CO7A1_HUMAN	sp|P49223|SPIT3_HUMAN	38.462	13	8	0	1425	1437	32	44	998	15.0
sp|P48307|TFPI2_HUMAN	sp|P02760|AMBP_HUMAN	23.810	21	16	0	164	184	72	92	997	11.5
sp|O43291|SPIT2_HUMAN	sp|O95428|PPN_HUMAN	40.000	15	9	0	60	74	178	192	996	11.5
sp|P12111|CO6A3_HUMAN	sp|O95428|PPN_HUMAN	71.429	7	2	0	2160	2166	983	989	995	15.4


Now you see that 

In [ ]:
#sp|P48307|TFPI2_HUMAN   sp|P05067|A4_HUMAN      27.273  11      8       0       154     164     551     561     999     11.5

This gives us information of the range of e-values we may get. 

If we are not interested in the full matches. We can use awk to check for the id's 
With the cmd below we get all the complete matches.

In [22]:
awk '{if($1 == $2)print $0}' Human_PF00014.bl6
#if the first index of a line is equal to the second, print the whole line ($0_)

sp|Q06481|APLP2_HUMAN	sp|Q06481|APLP2_HUMAN	100.000	763	0	0	1	763	1	763	0.0	1590
sp|P05067|A4_HUMAN	sp|P05067|A4_HUMAN	100.000	770	0	0	1	770	1	770	0.0	1610
sp|Q02388|CO7A1_HUMAN	sp|Q02388|CO7A1_HUMAN	100.000	2944	0	0	1	2944	1	2944	0.0	5494
sp|P12111|CO6A3_HUMAN	sp|P12111|CO6A3_HUMAN	100.000	3177	0	0	1	3177	1	3177	0.0	6455
sp|P02760|AMBP_HUMAN	sp|P02760|AMBP_HUMAN	100.000	352	0	0	1	352	1	352	0.0	734
sp|O95925|EPPI_HUMAN	sp|O95925|EPPI_HUMAN	100.000	133	0	0	1	133	1	133	8.42e-100	274
sp|P10646|TFPI1_HUMAN	sp|P10646|TFPI1_HUMAN	100.000	304	0	0	1	304	1	304	0.0	630
sp|O43278|SPIT1_HUMAN	sp|O43278|SPIT1_HUMAN	100.000	529	0	0	1	529	1	529	0.0	1099
sp|O43291|SPIT2_HUMAN	sp|O43291|SPIT2_HUMAN	100.000	252	0	0	1	252	1	252	0.0	521
sp|P48307|TFPI2_HUMAN	sp|P48307|TFPI2_HUMAN	100.000	235	0	0	1	235	1	235	2.53e-180	486
sp|Q8TEU8|WFKN2_HUMAN	sp|Q8TEU8|WFKN2_HUMAN	100.000	576	0	0	1	576	1	576	0.0	1199
sp|Q2UY09|COSA1_HUMAN	sp|Q2UY09|COSA1_HUMAN	100.000	1125	0	0	1	1125	1	1125	0.0	2187
sp|Q96NZ8|WFKN1_HUMAN	

In [23]:
sort -grk 11 autotest.bltab|head

sp|P48307|TFPI2_HUMAN	sp|P05067|A4_HUMAN	27.273	11	8	0	154	164	551	561	999	11.5
sp|P05067|A4_HUMAN	sp|P12111|CO6A3_HUMAN	22.222	45	33	1	251	295	1752	1794	999	13.5
sp|O95925|EPPI_HUMAN	sp|O43278|SPIT1_HUMAN	30.000	10	7	0	56	65	432	441	999	10.8
sp|Q8TEU8|WFKN2_HUMAN	sp|Q02388|CO7A1_HUMAN	37.500	16	10	0	97	112	2341	2356	998	13.1
sp|Q2UY09|COSA1_HUMAN	sp|Q02388|CO7A1_HUMAN	54.545	11	5	0	495	505	932	942	998	13.9
sp|Q2UY09|COSA1_HUMAN	sp|P12111|CO6A3_HUMAN	50.000	10	5	0	279	288	134	143	998	13.9
sp|Q02388|CO7A1_HUMAN	sp|P49223|SPIT3_HUMAN	38.462	13	8	0	1425	1437	32	44	998	15.0
sp|P48307|TFPI2_HUMAN	sp|P02760|AMBP_HUMAN	23.810	21	16	0	164	184	72	92	997	11.5
sp|O43291|SPIT2_HUMAN	sp|O95428|PPN_HUMAN	40.000	15	9	0	60	74	178	192	996	11.5
sp|P12111|CO6A3_HUMAN	sp|O95428|PPN_HUMAN	71.429	7	2	0	2160	2166	983	989	995	15.4


These are obviously the 18 proteins we have in our dataset.

In [24]:
awk '{if ($1 == $2)print $0}' Human_PF00014.bl6 |wc
#       18     216    1468

      18     216    1468


But we rather want to know what happened with the negatives. So lets blast the negatives.

In [25]:
blastp -query Human_NotPF00014.fasta -db Human_PF00014.fasta -out Human_NotPF00014_Training.bl6 -outfmt 6

We can less the file *.bl6
At this point we want to know the best hit between the negative, that are not Kunitz/BPTI and the Human Kunitz/BPTI. 
Now we sort the output to find the sequence with the lowest e-value in the 11th column.

In [ ]:
#sort -gk 11 Human_NotPF00014_Training.bl8 |less

Now we see something interesting here: there are 2 seq that we have retrieved from a Uniprot search that are not classified as Human BPTI but they have an e-value that is zero.
The first col is Human non BPTI and the second col is Human BPTI that we have in our dataset.


In [ ]:
sp|A6NMZ7|CO6A6_HUMAN   sp|P12111|CO6A3_HUMAN   32.023  2239    1310    43      27      2138    639     2792    0.0     1013
sp|A8TX70|CO6A5_HUMAN   sp|P12111|CO6A3_HUMAN   31.051  2235    1320    51      30      2130    639     2786    0.0     927

This means probably that those two (CO6A6_HUMAN, CO6A5_HUMAN) have a domain that is 
extremely similar to BPTI and has not yet been annotated by pfam.

Then there are many other hits that have an evalue of 10^-3 :
This number is also a possible threshold for selecting **homologues**

`
sp|A6NMZ7|CO6A6_HUMAN   sp|P12111|CO6A3_HUMAN   32.023  2239    1310    43      27      2138    639     2792    0.0     1013
sp|A8TX70|CO6A5_HUMAN   sp|P12111|CO6A3_HUMAN   31.051  2235    1320    51      30      2130    639     2786    0.0     927
sp|A0A075B6J6|LV322_HUMAN       sp|O95428|PPN_HUMAN     25.926  81      40      2       29      108     1053    1114    0.001   28.5
sp|A0A075B6J6|LV322_HUMAN       sp|O95428|PPN_HUMAN     33.750  80      35      4       29      108     919     980     0.001   28.5
sp|A0A075B6T6|TVAL2_HUMAN       sp|O95428|PPN_HUMAN     30.556  72      40      3       32      94      1142    1212    0.001   28.5
sp|A0A0J9YXQ4|PMA6E_HUMAN       sp|Q02388|CO7A1_HUMAN   30.182  275     119     8       86      293     2379    2647    0.001   32.3
sp|A0A1B0GUY1|MRCOL_HUMAN       sp|Q02388|CO7A1_HUMAN   25.397  189     126     5       106     282     2248    2433    0.001   31.2
sp|A0A1B0GUY1|MRCOL_HUMAN       sp|Q02388|CO7A1_HUMAN   33.333  51      34      0       206     256     1294    1344    0.001   31.2
sp|A2VEC9|SSPO_HUMAN    sp|O95428|PPN_HUMAN     36.585  82      41      4       2510    2584    313     390     0.001   35.8
sp|A6NDA9|LRIT2_HUMAN   sp|Q02388|CO7A1_HUMAN   30.851  94      59      4       359     452     864     951     0.001   32.0
sp|A6NF34|ANTRL_HUMAN   sp|Q02388|CO7A1_HUMAN   29.787  94      58      4       172     262     1157    1245    0.001   32.7
sp|B2RNN3|C1T9B_HUMAN   sp|Q02388|CO7A1_HUMAN   39.796  98      35      3       71      156     1296    1381    0.001   31.2
sp|O14498|ISLR_HUMAN    sp|O95428|PPN_HUMAN     29.630  54      38      0       294     347     1169    1222    0.001   31.6
sp|O15232|MATN3_HUMAN   sp|Q02388|CO7A1_HUMAN   23.529  136     98      4       102     234     1072    1204    0.001   32.0
sp|O15394|NCAM2_HUMAN   sp|O95428|PPN_HUMAN     23.333  210     96      8       222     382     921     1114    0.001   32.7
sp|O60271|JIP4_HUMAN    sp|P12111|CO6A3_HUMAN   25.000  88      63      1       487     574     1346    1430    0.001   33.9
sp|O60500|NPHN_HUMAN    sp|Q96NZ8|WFKN1_HUMAN   27.381  84      56      3       740     820     186     267     0.001   33.5
sp|O60575|ISK4_HUMAN    sp|Q8TEU8|WFKN2_HUMAN   40.909  22      13      0       46      67      138     159     0.001   27.7
sp|O60575|ISK4_HUMAN    sp|Q96NZ8|WFKN1_HUMAN   40.909  22      13      0       46      67      120     141     0.001   27.7
sp|O75056|SDC3_HUMAN    sp|O95428|PPN_HUMAN     64.706  17      6       0       405     421     1186    1202    0.001   31.6
sp|O75096|LRP4_HUMAN    sp|O43278|SPIT1_HUMAN   40.000  35      21      0       231     265     335     369     0.001   33.9
`

In [ ]:
sort -gk 11 Human_NotPF00014_Training.bl6 |head -n 2 |awk '{print $1,$2,$11}

Now we may see that there are many non BPTI/Kunitz proteins that get a low e-value. Moving to the training set we can now rund blast on both the positive and the negative sets.

In [3]:
# Run on positive testing set
# psiblast -query NotHuman_PF00014.fasta -db Human_PF00014.fasta -out NotHuman_PF00014.bl8
# didnt work on my version
# blastp -in FASTAIN -db DATABASE - out OUTFILE.bl6 -outfmt 6
blastp -query Human_PF00014.fasta -db Human_PF00014.fasta -out Human_PF00014.bl6 -outfmt 6

In [1]:
cat Human_PF00014.bl6 |wc -l

     835


`NotHuman_PF00014.bl8` contains the alignment of all non-human PF00014 proteins against the Human PF00014 sequences contained in `Human_PF00014.fasta`

Now we will run the same also for the human testing.

In [1]:
blastp -query NotHuman_PF00014.fasta -db Human_PF00014.fasta -out NotHuman_PF00014.bl6 -outfmt 6

In [2]:
cat Human_NotPF00014_2Training.bl6 |wc -l

  114207


`Human_NotPF00014_Testing.bl8` contains the alignment of the 1,000 sequences (Training set) of all human proteins `not belonging to PF00014` against the ones belonging to PF00014 family.


We need first to rank the output on the posotive set of non Human BPTI/Kunitz proteins. To do so we grep based on the identifiers and select the mathch with lowest e-value.

In [ ]:
# idk what he wants but this doesnt work for me:
#for i in `awk '{print $1}' NotHuman_PF00014.bl8 |sort -u ` ; do grep $i NotHuman_PF00014.bl8 |sort -gk 11 |head -n 1 ; done > NotHuman_PF00014.bl6.best

Let's select an e-value of 0.001 as a classification threshold for the classification of BPTI/kunitz proteins. Accorddin to this assumption we calculated the number of proteins with maximu e-value below that threshold. 

# Training
## Positive Training set

Now BLAST the positive training set not human but Kunitz:
Since it is slow I will comment it.

Then I sort it by the best E-value and print the first proteins few proteins via |head

In [29]:
#blastp -query NotHuman_PF00014.fasta -db Human_PF00014.fasta -out positive_training_test.bl6 -outfmt 6

In [30]:
sort -nrk 11 positive_training_test.bl6 |head


sp|P81906|ISPI2_GALME	sp|O95428|PPN_HUMAN	38.462	13	8	0	29	41	718	730	10.0	15.0
sp|Q9DA01|EPPI_MOUSE	sp|Q8IUA0|WFDC8_HUMAN	42.857	98	56	0	30	127	48	145	9.99e-23	81.3
sp|P15943|APLP2_RAT	sp|P10646|TFPI1_HUMAN	40.385	52	31	0	311	362	124	175	9.99e-09	48.1
sp|C0HJF4|VKT2_HETCR	sp|O95428|PPN_HUMAN	41.509	53	31	0	4	56	754	806	9.99e-13	51.6
sp|B6RLX2|VKTCT_OPHHA	sp|P02760|AMBP_HUMAN	43.137	51	29	0	31	81	287	337	9.99e-11	47.4
sp|B5L5Q1|VKT7_PSETT	sp|O43278|SPIT1_HUMAN	32.812	64	43	0	18	81	237	300	9.99e-09	41.6
sp|Q8AY44|VKTH7_BUNCA	sp|P05067|A4_HUMAN	34.783	46	30	0	36	81	296	341	9.98e-10	44.7
sp|Q75S50|VKTH3_BUNCA	sp|P05067|A4_HUMAN	34.783	46	30	0	36	81	296	341	9.98e-10	44.7
sp|P62757|IATR_SHEEP	sp|O43278|SPIT1_HUMAN	46.809	47	25	0	12	58	398	444	9.98e-11	48.9
sp|P62756|IATR_CAPHI	sp|O43278|SPIT1_HUMAN	46.809	47	25	0	12	58	398	444	9.98e-11	48.9


In [35]:
sort -gk 11 positive_training_test.bl6 |head

sp|P00978|AMBP_BOVIN	sp|P02760|AMBP_HUMAN	77.273	352	80	0	1	352	1	352	0.0	561
sp|P04366|AMBP_PIG	sp|P02760|AMBP_HUMAN	80.655	336	65	0	2	337	17	352	0.0	581
sp|P08592|A4_RAT	sp|P05067|A4_HUMAN	96.883	770	24	0	1	770	1	770	0.0	1557
sp|P08592|A4_RAT	sp|Q06481|APLP2_HUMAN	49.873	790	320	17	3	769	26	762	0.0	716
sp|P0C5J5|WFKN1_RAT	sp|Q8TEU8|WFKN2_HUMAN	56.691	538	209	8	31	550	41	572	0.0	607
sp|P0C5J5|WFKN1_RAT	sp|Q96NZ8|WFKN1_HUMAN	87.992	533	63	1	20	552	17	548	0.0	939
sp|P12023|A4_MOUSE	sp|P05067|A4_HUMAN	96.623	770	26	0	1	770	1	770	0.0	1551
sp|P12023|A4_MOUSE	sp|Q06481|APLP2_HUMAN	50.253	790	317	17	3	769	26	762	0.0	721
sp|P15943|APLP2_RAT	sp|Q06481|APLP2_HUMAN	92.318	742	51	2	26	765	26	763	0.0	1340
sp|P15989|CO6A3_CHICK	sp|P12111|CO6A3_HUMAN	65.153	2884	977	11	1	2873	1	2867	0.0	3726


For each protein in the query, Saul wants to find the best E-value possible. He created an awk array with the first column as index, so to collapse duplicates and store the best E-value. I output an identical bltab file but filtered to have just the best match for each sequence. The filtered output contains 341 alignments, which is the same number of the number of unique indentifiers in the original file.

In [37]:
cat positive_training_test.bl6|awk \
'{\
query=$1;\
if (!(query in out) || $11 < evalue[query])\
    {evalue[query]=$11; out[query]=$0}\
}\
END{for(key in out){print out[key]}}'\
> positive_training_test_topscores.bl6

In [38]:
cat positive_training_test_topscores.bl6|sort -rgk 11|head # worst best scores for each sequence
wc -l positive_training_test_topscores.bl6
cat positive_training_test_topscores.bl6|awk '{print $1}'|sort -u|wc -l #number of unique queries in the original BLAST output

sp|D3GGZ8|BLI5_HAECO	sp|Q2UY09|COSA1_HUMAN	23.944	71	41	4	125	191	1062	1123	0.045	25.0
sp|O62247|BLI5_CAEEL	sp|Q96NZ8|WFKN1_HUMAN	29.167	72	48	2	115	184	339	409	2.44e-04	32.0
sp|Q11101|YL15_CAEEL	sp|P05067|A4_HUMAN	37.500	40	25	0	38	77	305	344	1.85e-07	41.2
sp|P86963|KCP2_PINMA	sp|P12111|CO6A3_HUMAN	36.207	58	30	2	98	153	3112	3164	1.33e-07	43.9
sp|P0CAR0|VKTG1_MICPY	sp|P12111|CO6A3_HUMAN	44.828	29	16	0	6	34	3112	3140	6.47e-08	37.0
sp|P0CH75|VKT25_CYRHA	sp|P12111|CO6A3_HUMAN	44.118	34	17	1	2	33	3110	3143	1.01e-09	42.0
sp|Q2ES49|VKT2_DABRR	sp|P10646|TFPI1_HUMAN	38.596	57	35	0	25	81	119	175	6.96e-10	44.7
sp|G9I929|VKTA_MICTN	sp|P10646|TFPI1_HUMAN	35.484	62	35	2	24	83	118	176	6.31e-10	45.1
sp|C0LNR2|VKT_VESBI	sp|O95428|PPN_HUMAN	38.710	62	33	2	20	76	743	804	9.51e-11	47.0
sp|Q7Z1K3|ANIS1_ANISI	sp|Q02388|CO7A1_HUMAN	41.558	77	34	4	28	101	2876	2944	5.74e-11	52.0
     341 positive_training_test_topscores.bl6
     341


## Summary of positive training
The worst top score for a sequence in the positive training set is 0.045, followed by e-04 and e-07.
In total I have 341 unique sequences in the query.

## Negative training set
Now I BLAST the negative training set (human not Kunitz training).
It is slow, don't run just for fun. This is why I am commenting it.
I sort by best E-value and print the first proteins.
I need to set an high E-value threshold otherwise I don't get an hit for all the sequences.
However, even with a 1000 thershold I am still missing 2 sequences (9998 unique hits).
I will never be able to compute with an higher threshold.

In [41]:
blastp -query Human_NotPF00014_Training.fasta -db Human_PF00014.fasta -out negative_training.bl6 -outfmt 6 -evalue 100 

The first 2 sequences are strange. They have 0 E-value and their names seem to be really similar.

For each protein in the query, I want to find the best E-value possible. I create an awk array with the first column as index, so to collapse duplicates and store the best E-value. I output an identical bltab file but filtered to have just the best match for each sequence. The filtered output contains 9998 alignments, which is the same number of the number of unique indentifiers in the original file.

In [ ]:
cat negative_training.bl6|awk \
'{\
query=$1;\
if (!(query in out) || $11 < evalue[query])\
    {evalue[query]=$11; out[query]=$0}\
}\
END{for(key in out){print out[key]}}'\
> negative_training_topscores.bltab